In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

In [4]:
import torch
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from optimizers_impl.gsam import GSAM, CosineScheduler, ProportionScheduler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [5]:
def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        def loss_fn(predictions, targets):
            return smooth_crossentropy(predictions, targets, smoothing=0.1).mean()

        optimizer.set_closure(loss_fn, inputs, targets)
        predictions, loss = optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler.step()
            optimizer.update_rho_t()


In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())


In [7]:
# if __name__ == "__main__":

#     num_epochs = 50
#     model_name = '/content/drive/MyDrive/ColabNotebooks/results_logs/widerestnet-gsam'
#     # model_name = 'widerestnet-gsam'
#     initialize(seed=111)
#     log = Log(log_each=1, log_name=model_name)

#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#     model = WideResNet(16, 8, dropout=0.0, in_channels=3, labels=2).to(device)

#     base_optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.6, momentum=0.55)
#     dataset = Coronary(batch_size=100, threads=2, img_size_definition=32)

#     scheduler = CosineScheduler(T_max=num_epochs * len(dataset.train), max_value=0.01, min_value=0.0, optimizer=base_optimizer)
#     rho_scheduler = ProportionScheduler(pytorch_lr_scheduler=scheduler, max_lr=0.001, min_lr=0.0, max_value=0.1, min_value=0.0)
#     optimizer = GSAM(params=model.parameters(), base_optimizer=base_optimizer, model=model, gsam_alpha=0.0001, rho_scheduler=rho_scheduler, adaptive=True)

#     for epoch in range(num_epochs):
#         train(model, device, dataset.train, optimizer, log, scheduler)
#         test(model, device, dataset.valid, log)

#     log.flush()


In [8]:
if __name__ == "__main__":

    num_epochs = 50
    model_name = '/content/drive/MyDrive/ColabNotebooks/results_logs/widerestnet-gsam-75'
    # model_name = 'widerestnet-gsam-75'
    initialize(seed=111)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = torch.hub.load('pytorch/vision:v0.10.0', 'wide_resnet50_2', pretrained=True)
    in_features = model.fc.in_features
    model.fc = torch.nn.Linear(in_features, 2)
    model.to(device)
    model.eval()
    # model.classifier[1] = nn.Linear(1280, 2)


    base_optimizer = torch.optim.SGD(model.parameters(), lr=0.003, weight_decay=0.2, momentum=0.5)
    dataset = Coronary(batch_size=100, threads=2, img_size_definition=75)

    scheduler = CosineScheduler(T_max=num_epochs * len(dataset.train), max_value=0.0015, min_value=0.0, optimizer=base_optimizer)
    rho_scheduler = ProportionScheduler(pytorch_lr_scheduler=scheduler, max_lr=0.001, min_lr=0.0, max_value=0.1, min_value=0.0)
    optimizer = GSAM(params=model.parameters(), base_optimizer=base_optimizer, model=model, gsam_alpha=0.3, rho_scheduler=rho_scheduler, adaptive=True)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.3708  │     53.11 %  ┃   1.499e-03  │       00:26  ┃      0.3938  │     41.67 %  ┃
┃           1  ┃      0.2931  │     71.22 %  ┃   1.495e-03  │       00:06  ┃      0.3831  │     50.00 %  ┃
┃           2  ┃      0.2376  │     83.44 %  ┃   1.489e-03  │       00:06  ┃      0.3801  │     53.33 %  ┃
┃           3  ┃      0.2085  │     85.89 %  ┃   1.479e-03  │       00:06  ┃      0.3781  │     59.67 %  ┃
┃           4  ┃      0.1764  │     89.22 %  ┃   1.466e-03  │       00:06  ┃      0.3723  │     56.67 %  ┃
┃           5  ┃      0.1545  │     90.67 %  ┃   1.451e-03  │       00:06  ┃      0.3685  │     60.00 %  ┃
┃           6  ┃      0.1289  │     93.56 %  ┃   1.433e-03  │       00:06  ┃      0.3157  │     63.67 %  ┃
┃           7  ┃      0.1056  │     94.78 %  ┃   1.412e-03  │       00:06  ┃      0.3005  │     68.67 %  ┃
┃           8  ┃      0.0989  │     94.78 %  ┃   1.389e-03  │       00:06  ┃      0.2879  │     71.00 %  ┃
┃           9  ┃      0.0825  │     9

In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [10]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)



tensor(0.9709, device='cuda:0')
tensor(0.9700, device='cuda:0')
tensor(0.9434, device='cuda:0')
tensor(0.9400, device='cuda:0')
tensor(1., device='cuda:0')
